In [1]:
import glob #to read the files
import json #to work if .json
import numpy as np #to math
import pandas as pd #to save the data
import math #to convertion calculus
from astropy.time import Time #to time converting
from astropy import units #time correction
from astropy.coordinates import SkyCoord #time correction
from scipy import interpolate #to interpolate the wavelength and flux
from tabulate import tabulate #to export in table format
from scipy.interpolate import interp1d
#import scipy.optimize as opt
from scipy import optimize
from scipy import signal
import os.path
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from scipy import stats
import os.path 
import sncosmo

In [2]:
save_path = '/home/joao/Documentos/templates'

In [3]:
kernel = 10.0 * RBF(length_scale=20.0, length_scale_bounds=(5, 25)) + WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-10, 1e+1))

In [4]:
csp = sncosmo.get_magsystem('csp')
ab = sncosmo.get_magsystem('ab')
vega = sncosmo.get_magsystem('vega')

In [5]:
%run ./functions.ipynb

In [6]:
hist = []
t0_values = []
max_namesV = []
max_namesr = []

In [7]:
sn_names = glob.glob("*.json")

In [8]:
len(sn_names)

262

In [9]:
maxvaluesV = []
maxvaluesr = []

In [10]:
for j in range(0,len(sn_names)):  
    
    name = sn_names[j]

    print(name)
    
    with open(name, "r") as read_file:
        data = json.load(read_file)

    url = name
    if url.endswith('.json'):
        url = url[:-5]

    redshift = redshift_value(data[url])

    if redshift is not None:
        
        magV = []
        timeV = []
        systemV = []
        
        magr = []
        timer = []
        systemr = []

        for i in range(0,len(data[url]["photometry"])):

            if "band" in data[url]["photometry"][i]:

                if data[url]["photometry"][i]["band"] == "V":
                    
                    if "system" in data[url]["photometry"][i]:
                        
                        systemV.append(data[url]["photometry"][i]["system"])
                        magV.append(float(data[url]["photometry"][i]["magnitude"]))
                        timeV.append(float(data[url]["photometry"][i]["time"]))
                    
                    else:
                        
                        systemV.append("w")
                        magV.append(float(data[url]["photometry"][i]["magnitude"]))
                        timeV.append(float(data[url]["photometry"][i]["time"]))
                        
                if data[url]["photometry"][i]["band"] == "v":
                    
                    if "system" in data[url]["photometry"][i]:
                        
                        systemV.append(data[url]["photometry"][i]["system"])
                        magV.append(float(data[url]["photometry"][i]["magnitude"]))
                        timeV.append(float(data[url]["photometry"][i]["time"]))
                    
                    else:
                        
                        systemV.append("w")
                        magV.append(float(data[url]["photometry"][i]["magnitude"]))
                        timeV.append(float(data[url]["photometry"][i]["time"]))
                        
                        
                        
                        
                        
                if data[url]["photometry"][i]["band"] == "R":
                    
                    if "system" in data[url]["photometry"][i]:
                        
                        systemr.append(data[url]["photometry"][i]["system"])
                        magr.append(float(data[url]["photometry"][i]["magnitude"]))
                        timer.append(float(data[url]["photometry"][i]["time"]))
                    
                    else:
                        
                        systemr.append("w")
                        magr.append(float(data[url]["photometry"][i]["magnitude"]))
                        timer.append(float(data[url]["photometry"][i]["time"]))
                        
                        
                if data[url]["photometry"][i]["band"] == "r":
                    
                    if "system" in data[url]["photometry"][i]:
                        
                        systemr.append(data[url]["photometry"][i]["system"])
                        magr.append(float(data[url]["photometry"][i]["magnitude"]))
                        timer.append(float(data[url]["photometry"][i]["time"]))
                    
                    else:
                        
                        systemr.append("w")
                        magr.append(float(data[url]["photometry"][i]["magnitude"]))
                        timer.append(float(data[url]["photometry"][i]["time"]))
                                                
        
        if len(timeV) > 3:
    
            #maxvalues.append(min(mag))
            
            ntimeV = []
            nmagV = []
            nsysV = []
            
            ntimer = []
            nmagr = []
            nsysr = [] 
            
            for i in range(0,len(magV)):

                if np.abs(timeV[i]- min(timeV)) < 100:
                        
                    if systemV[i] == "CSP":                        
                        
                        nmagV.append(csp.band_mag_to_flux(magV[i], 'cspv9844')*(1+redshift))
                        ntimeV.append([timeV[i]])
                        
                    if systemV[i] == "AB":
                        #print("ab")
                        
                        nmagV.append(ab.band_mag_to_flux(magV[i], 'cspv9844')*(1+redshift))
                        ntimeV.append([timeV[i]])
                        
                    if systemV[i] == "Vega":
                        
                        #print("vega")
                        nmagV.append(vega.band_mag_to_flux(magV[i], 'cspv9844')*(1+redshift))
                        ntimeV.append([timeV[i]])   
                        
                    if systemV[i] == "w":
                        
                        #print("w")
                        nmagV.append(csp.band_mag_to_flux(magV[i], 'cspv9844')*(1+redshift))
                        ntimeV.append([timeV[i]])                     

                        

            for i in range(0,len(magr)):

                if np.abs(timer[i]- min(timer)) < 100:
                        
                    if systemr[i] == "CSP":                        
                        
                        nmagr.append(csp.band_mag_to_flux(magr[i], 'cspr')*(1+redshift))
                        ntimer.append([timer[i]])
                        
                    if systemr[i] == "AB":
                        #print("ab")
                        
                        nmagr.append(ab.band_mag_to_flux(magr[i], 'cspr')*(1+redshift))
                        ntimer.append([timer[i]])
                        
                    if systemr[i] == "Vega":
                        
                        #print("vega")
                        nmagr.append(vega.band_mag_to_flux(magr[i], 'cspr')*(1+redshift))
                        ntimer.append([timer[i]])   
                        
                    if systemr[i] == "w":
                        
                        #print("w")
                        nmagr.append(csp.band_mag_to_flux(magr[i], 'cspr')*(1+redshift))
                        ntimer.append([timer[i]])                     
                                      
                    #print(np.abs(time[i][0] - min(time[i])))    
                    
                    
                
                
            if len(nmagV) > 3:
                
                gpV = GaussianProcessRegressor(kernel=kernel, alpha=0.0).fit(ntimeV, nmagV)

                X_V = np.linspace(min(ntimeV)[0], max(ntimeV)[0], 1000)
                #print(X_)
                y_meanV = gpV.predict(X_V[:, np.newaxis])

                y_meanV = list(y_meanV)

                idxV = y_meanV.index(max(y_meanV))  
                
                
                if idxV > 2: 
                    
                    maxvaluesV.append(max(y_meanV))
                    t0_values.append([X_V[idxV], sn_names[j]])
                    max_namesV.append(sn_names[j])
                            
                
                
            if len(nmagr) > 3:
                
                gpr = GaussianProcessRegressor(kernel=kernel, alpha=0.0).fit(ntimer, nmagr)

                X_r = np.linspace(min(ntimer)[0], max(ntimer)[0], 1000)
                #print(X_)
                y_meanr = gpr.predict(X_r[:, np.newaxis])

                y_meanr = list(y_meanr)

                idxr = y_meanr.index(max(y_meanr))  
                
                
                if idxr > 2: 
                    
                    maxvaluesr.append(max(y_meanr))
                    max_namesr.append(sn_names[j])          

SN2008bf.json
0.024027
SN2007ca.json
0.014066
SN2006hb.json
0.0153328042114802
SN2006gj.json
0.02829
SN2006gz.json
None
PTF11bui.json
0.02846
SN2005am.json
0.007899
SN2006et.json
0.02217798925424974
SN2002ck.json
0.029864
SN1998es.json
0.01057
SN2008ec.json
0.016317
SN1995bd.json
None
SN2012dn.json
0.010187
SN2013cs.json
0.009
SN1995E.json
0.01158
SN2002cd.json
0.010344
SN2002bo.json
0.00424
SN2008fp.json
0.005656012429203772
SN1992A.json
0.00626
SN2001N.json
0.021025
SN2008ae.json
0.03006
SN2007bm.json
0.006211
SN2014dt.json
0.0052
SN2007F.json
0.023591
SN2003W.json
0.020071
SN2006bt.json
0.032156
SN2012ht.json
0.003559
SN2006kf.json
0.02129693315973835
SN2007bj.json
0.016708
SN2007co.json
0.026962
LSQ12gdj.json
0.03
SN1995ak.json
0.02272
SN1997cn.json
0.0162
SN2014J.json
0.000841
SN2008hv.json
0.012526589416171241
SN2005kc.json
0.01512
SN2001V.json
0.01502
SN1991bg.json
0.003539


/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


SN2006cj.json
0.067517
SN1972E.json
0.001358
SN2001gc.json
0.01929
SN2002fb.json
0.015567
SN2003iv.json
0.034307
SN1993Z.json
0.00452
SN2007N.json
0.012879
SN2005ke.json
0.00488
SN2001E.json
0.01926
SN2002dj.json
0.009393
SN1997br.json
0.006941
SN2007fb.json
0.018026
SN2005ki.json
0.019207


/home/joao/.local/lib/python3.8/site-packages/sklearn/gaussian_process/_gpr.py:494: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


SN2007af.json
0.005464
SN2002ha.json
0.014046
SN2006lf.json
0.013189
SN2010ae.json
0.003662
SN2011fe.json
0.000804
SN2006dm.json
0.022019
SN1957B.json
0.00659
SN1980N.json
0.005871
SN2008dr.json
0.041449
SN2001el.json
0.00536
SN2007hj.json
0.01411
SN2008J.json
0.015874
SN2003du.json
0.006381
SN2004ey.json
0.01579444768273275
SN2005eq.json
0.028974294942283096
SN2006oa.json
redshift nao encontrado
SN2006ev.json
0.02873
SN2007ax.json
0.006861
SN2008ar.json
0.026147
SN2010cr.json
0.02157
PTF10icb.json
0.008544
SN2005hk.json
0.01306
SN2004gs.json
0.02660272499077032
SN2017hbi.json
0.04
SN1998V.json
0.01759
SN1992G.json
0.00529
SN2003it.json
0.025121
SN2009ag.json
0.00864
SN2006X.json
0.00524
SN2005el.json
0.01490009610926979
SN1991M.json
0.007235
SN2004S.json
0.00937
SN1989B.json
0.002425
SN2017erp.json
0.006174
SN1998dx.json
0.054221
SN1998bp.json
0.01043
SN2006ax.json
0.016725
SN2003fa.json
0.006004
SN2003cg.json
0.00413
SN2008dt.json
0.03539
SN2002kf.json
0.0193
SN2008ds.json
0.02103
SN

In [11]:
maxvaluesV

[0.27559141438715906,
 0.2747320282883096,
 0.05464090740545802,
 1.8575515541282437,
 0.27473420669537774,
 0.3050543131658736,
 1.7162640499080555,
 0.3413447273167561,
 1.140578229925552,
 1.7779353073834694,
 0.2467743298011576,
 0.19061881578778253,
 2.073182179143149,
 1.5759191308237632,
 5.353415637720821,
 0.17804142516035815,
 0.03422593857925863,
 1.2830430569275963,
 0.2511823223759795,
 0.27084433988015816,
 0.10309586920201667,
 3.32288396263691,
 0.13488315834488035,
 0.12483495164500792,
 0.2508600739261926,
 32.16280923800781,
 0.6897073531408413,
 0.29928310814482834,
 0.9112462487189672,
 1.5016752209056676,
 0.13054787952685043,
 0.07648573035275086,
 1.248638938248579,
 0.165615182515844,
 1.333562241990947,
 1.8900662900055565,
 0.2863100822748254,
 0.3209639111627194,
 3.1054771718618364,
 0.5644972930692482,
 0.10528697482435426,
 59.54031210357061,
 0.22040263372920918,
 6.334218791490715,
 0.05721558217354311,
 4.86500767217953,
 0.32295809649474444,
 0.275357

In [12]:
maximosV = []
maximosr = []
nome = []

In [13]:
for i in range(0, len(sn_names)):
    
    if sn_names[i] in max_namesV:
    
        if sn_names[i] in max_namesr:

            nome.append(sn_names[i])
            
            idx_V = max_namesV.index(sn_names[i])

            maximosV.append(maxvaluesV[idx_V])
        
            idx_r = max_namesr.index(sn_names[i])

            maximosr.append(maxvaluesr[idx_r])
        
    if sn_names[i] in max_namesV:
    
        if sn_names[i] not in max_namesr:
            
            nome.append(sn_names[i])
            
            idx_V = max_namesV.index(sn_names[i])

            maximosV.append(maxvaluesV[idx_V])

            maximosr.append(-9999)
            
            
            
    if sn_names[i] not in max_namesV:
    
        if sn_names[i] in max_namesr:
            
            nome.append(sn_names[i])
            
            idx_r = max_namesr.index(sn_names[i])

            maximosr.append(maxvaluesr[idx_r])

            maximosV.append(-9999)

In [14]:
maximosV

[0.27559141438715906,
 0.2747320282883096,
 0.05464090740545802,
 1.8575515541282437,
 0.27473420669537774,
 0.3050543131658736,
 1.7162640499080555,
 0.3413447273167561,
 1.140578229925552,
 1.7779353073834694,
 0.2467743298011576,
 0.19061881578778253,
 2.073182179143149,
 1.5759191308237632,
 5.353415637720821,
 0.17804142516035815,
 0.03422593857925863,
 1.2830430569275963,
 0.2511823223759795,
 0.27084433988015816,
 0.10309586920201667,
 3.32288396263691,
 0.13488315834488035,
 0.12483495164500792,
 0.2508600739261926,
 32.16280923800781,
 0.6897073531408413,
 0.29928310814482834,
 0.9112462487189672,
 1.5016752209056676,
 -9999,
 0.13054787952685043,
 -9999,
 0.07648573035275086,
 1.248638938248579,
 0.165615182515844,
 1.333562241990947,
 1.8900662900055565,
 0.2863100822748254,
 0.3209639111627194,
 3.1054771718618364,
 0.5644972930692482,
 0.10528697482435426,
 59.54031210357061,
 0.22040263372920918,
 6.334218791490715,
 0.05721558217354311,
 4.86500767217953,
 0.322958096494

In [15]:
names = []
zerotime = []

In [16]:
for i in range(0,len(t0_values)):
    
    zerotime.append(t0_values[i][0])
    names.append(t0_values[i][1][:-5])

In [17]:
names

['SN2008bf',
 'SN2007ca',
 'SN2006gj',
 'SN2005am',
 'SN2006et',
 'SN2002ck',
 'SN1998es',
 'SN2008ec',
 'SN2012dn',
 'SN2013cs',
 'SN1995E',
 'SN2002cd',
 'SN2002bo',
 'SN2008fp',
 'SN1992A',
 'SN2001N',
 'SN2008ae',
 'SN2007bm',
 'SN2007F',
 'SN2003W',
 'SN2006bt',
 'SN2012ht',
 'SN2006kf',
 'SN2007co',
 'LSQ12gdj',
 'SN2014J',
 'SN2008hv',
 'SN2005kc',
 'SN2001V',
 'SN1991bg',
 'SN2002fb',
 'SN2007N',
 'SN2005ke',
 'SN2001E',
 'SN2002dj',
 'SN1997br',
 'SN2007fb',
 'SN2005ki',
 'SN2007af',
 'SN2002ha',
 'SN2006lf',
 'SN2011fe',
 'SN2006dm',
 'SN1980N',
 'SN2008dr',
 'SN2001el',
 'SN2007hj',
 'SN2008J',
 'SN2003du',
 'SN2004ey',
 'SN2005eq',
 'SN2007ax',
 'SN2008ar',
 'PTF10icb',
 'SN2005hk',
 'SN2004gs',
 'SN2017hbi',
 'SN2003it',
 'SN2006X',
 'SN2005el',
 'SN1989B',
 'SN2017erp',
 'SN1998bp',
 'SN2006ax',
 'SN2003fa',
 'SN2003cg',
 'SN2008ds',
 'SN2006mr',
 'SN2001br',
 'SN1999aa',
 'SN2005cf',
 'SN2004at',
 'SN1995D',
 'SN2002eu',
 'SN2006D',
 'SN1998aq',
 'SN2005A',
 'SN2007on',


In [18]:
for i in range(0, len(nome)):
    
    nome[i] = nome[i][:-5]

In [19]:

table = []
for l in range(0,len(names)):
    table.append((names[l],zerotime[l]))

name_of_file = "t0_values"

completeName = os.path.join(save_path, name_of_file+".txt")

f = open(completeName, 'w')
f.write(tabulate(table, tablefmt="plain"))
f.close()


In [20]:

table = []
for l in range(0,len(nome)):
    table.append((nome[l], maximosV[l], maximosr[l]))

name_of_file = "max_values"

completeName = os.path.join(save_path, name_of_file+".txt")

f = open(completeName, 'w')
f.write(tabulate(table, tablefmt="plain"))
f.close()